In [1]:
import numpy as np
%load_ext Cython

In [2]:
"""
Most of the BJ stuff should follow general game conventions. 
We initially model more intricate details (burning, shuffling protocol) based on local casino.
https://www.crownmelbourne.com.au/getsydmedia/99be12f2-c48b-44dc-b6ba-ab5849f6f723/crown-melbourne-gaming-blackjack-rules_1.pdf?ext=.pdf
https://en.wikipedia.org/wiki/Glossary_of_blackjack_terms
"""

# configurations
NDECKS: int = 8
# NPLAYERS: int = 4 # affects how many cards you may be able to count
DECK_PENETRATION: int = 2 # typical number of decks dealt out before reshuffle

In [3]:
"""
We keep our representation of the deck and cards as characters, rather than grouping on BJ value
Importantly, this means our framework remains flexible enough to add in weird house rules such as perfect pairs/flushes
"""
cards = np.array(['2','3','4','5','6','7','8','9','T','J','Q','K','A']) # use char T for 10 
suits = np.array(['D', 'S', 'C', 'H'])
deck = np.array([np.char.add(cards, suit) for suit in suits]).ravel()
shoe = np.repeat(deck, NDECKS)

# We use bytestrings to enable Cython memview of C char[2] dtypes
shoe = shoe.astype('S2')

numcardsdeal = len(deck) * DECK_PENETRATION

In [4]:
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
%timeit rng.shuffle(shoe)

8.67 µs ± 546 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
# bad idea to use value mapping after every shuffle
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(cards.astype('S1'), values))
%timeit np.array([dict_map[x[:1]] for x in shoe]) # slicing byte string to get first char
# %timeit np.vectorize(dict_map.get)(shoe)

64.8 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
# may change to use numpy to read-in csv, to avoid pandas dependency
import pandas as pd
strat = pd.read_csv(r"StrategyInput\BasicNoDeviations-4to8Decks-HitSoft17.csv", header=None)

"""
Read in the individual strategy tables, and set top left corner position to 0.
"""
hard = strat.iloc[0:18, :].copy().reset_index(drop=True)
hard[0][0] = '0'
hard = hard.values.astype(str)
hrd = hard[1:,1:].astype('S1')

soft = strat.iloc[18:27, :].copy().reset_index(drop=True)
soft[0][0] = '0'
soft = soft.values.astype(str)
sft = soft[1:,1:].astype('S1')

splits = strat.iloc[27:38, :].copy().reset_index(drop=True)
splits[0][0] = '0'
splits = splits.values.astype(str)
splts = splits[1:,1:].astype('S1')

In [14]:
count = pd.read_csv(r"StrategyInput\HiLoCount.csv", header=None)
cnt = count.iloc[:,1].values.astype(float)

In [41]:
%%cython --cplus
import numpy as np
cimport cython 
from libcpp.unordered_map cimport unordered_map # we use the map from the C++ STL - quicker than Python dict

ctypedef unordered_map[char, Py_ssize_t] parse_char_t # create a mapping type for our char bytes
ctypedef unordered_map[int, Py_ssize_t] get_idx_t # get indexes based on hard values

# Create a packed struct in C representing one card in the shoe
# https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
cdef packed struct card_dtype_struct: 
    char face
    char suit

# C uses ascii integer representation for characters
# Numeric characters in our shoe will have an ascii value less than ascii of 'a'
# English characters will have an ascii value greater than ascii of 'a'
# https://www.cs.cmu.edu/~pattis/15-1XX/common/handouts/ascii.html
cdef int ASCII_A = 65 

# Map dealer upcard to vertical (second dimension - C order) index
cdef parse_char_t dealer_idx
dealer_idx.insert({'2', 0})
dealer_idx.insert({'3', 1})
dealer_idx.insert({'4', 2})
dealer_idx.insert({'5', 3})
dealer_idx.insert({'6', 4})
dealer_idx.insert({'7', 5})
dealer_idx.insert({'8', 6})
dealer_idx.insert({'9', 7})
dealer_idx.insert({'T', 8})
dealer_idx.insert({'J', 8})
dealer_idx.insert({'Q', 8})
dealer_idx.insert({'K', 8})
dealer_idx.insert({'A', 9})

# function decorators to turn off various things, in the interest of speed
# https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
def play_shoe(
    card_dtype_struct[:] shoe not None, 
    int numcardsdeal, 
    # We make sure our views have C contiguous order
    # https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
    char[:,::1] hrd not None, 
    char[:,::1] sft not None, 
    char[:,::1] splt not None, 
    double[:] cnt not None):
    """
    Simulate playing one shuffled shoe, until a cut card is drawn. 
    For now, return to main Python loop for reshuffle. 

    Inputs: (REFER TO TEMPLATE FILES IN STRATEGY INPUT)
        - 'shoe': numpy array (buffer) of char[2] byte strings
        - 'numcardsdeal': int, simulates the depth of the cut card
        - 'hrd': numpy array (buffer) of char bytes, in the input format dictating BJ action on player hard hand
        - 'sft': numpy array (buffer) of char bytes, in the input format dictating BJ action on player soft hand
        - 'splt': numpy array (buffer) of char bytes, in the input format dictating BJ action on player paired hand
        - 'cnt': numpy array (buffer) of double (Python float6), in the input format, with count values
    """
    cdef Py_ssize_t dealt = 0 # track what has been 'dealt' from shoe
    cdef double count = 0 # card count
    
    

    # this should roughly simulate a 'cut card'
    # we have to exceed the cut position prior to dealing
    # while dealt < numcardsdeal:
        




TypeError: 'str' object cannot be interpreted as an integer

In [35]:
hard

array([['0', '2', '3', '4', '5', '6', '7', '8', '9', 'T', 'A'],
       ['4', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['5', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['6', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['7', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['8', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['9', 'H', 'D', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
       ['10', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'H', 'H'],
       ['11', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'],
       ['12', 'H', 'H', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['13', 'S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['14', 'S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['15', 'S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'R', 'R'],
       ['16', 'S', 'S', 'S', 'S', 'S', 'H', 'H', 'R', 'R', 'R'],
       ['17', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'R'],
       ['18', 'S', 'S', 'S', 'S'

In [20]:
play_shoe(shoe, numcardsdeal, hrd, sft, splts, cnt)

1.0


In [ ]:
shoe[:50]

array([b'QS', b'9S', b'8S', b'TS', b'9C', b'JS', b'4S', b'5H', b'6S',
       b'6S', b'8C', b'JH', b'5H', b'2H', b'7H', b'JC', b'QS', b'6H',
       b'KH', b'7C', b'JS', b'5D', b'KD', b'2H', b'9C', b'4H', b'AS',
       b'KC', b'6C', b'6S', b'QC', b'8D', b'4D', b'5C', b'7H', b'9D',
       b'QH', b'2H', b'7S', b'TD', b'TS', b'5C', b'8D', b'KS', b'7C',
       b'AC', b'9D', b'TH', b'JS', b'8D'], dtype='|S2')

In [21]:
cnt

array([ 1.,  1.,  1.,  1.,  1.,  0.,  0.,  0., -1., -1., -1., -1., -1.])